In [1]:
import numpy as np

In [2]:
model_weight = [0.9098, 0.9113, 0.9111]

In [3]:
def mean(weight):
    weight = np.array(weight)
    weight = weight / sum(weight)
    return weight

In [4]:
mean(model_weight)

array([0.33299173, 0.33354074, 0.33346754])